In [0]:
%matplotlib inline

In [0]:
!pip install git+https://github.com/fastai/fastai.git
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"iamkartik","key":"a723f8654b75ed6d2891131c0b2d3218"}'}

In [3]:
!mkdir ~/.kaggle
!cp /content/kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

!kaggle competitions download -c plant-seedlings-classification

  0% 0.00/5.13k [00:00<?, ?B/s]
100% 5.13k/5.13k [00:00<00:00, 2.46MB/s]
 86% 74.0M/86.0M [00:01<00:00, 54.4MB/s]
100% 86.0M/86.0M [00:01<00:00, 78.9MB/s]
 99% 1.59G/1.60G [00:14<00:00, 155MB/s]
100% 1.60G/1.60G [00:14<00:00, 121MB/s]


In [38]:
!ls

kaggle.json  sample_submission.csv.zip	test.zip  train.zip
sample_data  test			train


In [47]:
#!unzip -q test.zip
#!unzip -q train.zip
!unzip sample_submission.csv.zip

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   


In [6]:
!ls

kaggle.json  sample_submission.csv.zip	test.zip  train.zip
sample_data  test			train


In [0]:
from fastai.imports import *
from fastai.conv_learner import *
from fastai.sgdr import *
from fastai.learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.plots import *
from fastai.transforms import *
import os
import pandas as pd
import numpy as np

In [0]:
train_data = os.listdir('train')
test_data = os.listdir('test')

In [13]:
len(test_data)

794

In [14]:
#train data already subdivided in folders 
train_data[:12]

['Fat Hen',
 'Loose Silky-bent',
 'Black-grass',
 'Common wheat',
 'Small-flowered Cranesbill',
 'Shepherds Purse',
 'Charlock',
 'Common Chickweed',
 'Sugar beet',
 'Scentless Mayweed',
 'Cleavers',
 'Maize']

In [113]:
plants=dict()

for data in train_data:
  plants[data]=os.listdir(f'train/{data}')

plength =dict()
for data in plants:
  plength[data] = len(plants[data])
  
plength

{'Black-grass': 185,
 'Charlock': 273,
 'Cleavers': 201,
 'Common Chickweed': 428,
 'Common wheat': 155,
 'Fat Hen': 333,
 'Loose Silky-bent': 458,
 'Maize': 155,
 'Scentless Mayweed': 362,
 'Shepherds Purse': 162,
 'Small-flowered Cranesbill': 348,
 'Sugar beet': 270}

In [0]:
import shutil
shutil.rmtree('valid')
os.mkdir('valid')

for data in train_data:
  os.mkdir(f'valid/{data}')
  i=0
  maxcounter = int(np.floor(len(plants[data])*.3))
  for image in plants[data]:
    if i==maxcounter:
      break
    shutil.move(f'train/{data}/{image}',f'valid/{data}/{image}') 
    i+=1

In [41]:
valid_data = os.listdir('valid')
valid_data

['Fat Hen',
 'Loose Silky-bent',
 'Black-grass',
 'Common wheat',
 'Small-flowered Cranesbill',
 'Shepherds Purse',
 'Charlock',
 'Common Chickweed',
 'Sugar beet',
 'Scentless Mayweed',
 'Cleavers',
 'Maize']

In [112]:
vdict = dict()
for data in valid_data:
  vdict[data]=len(os.listdir(f'valid/{data}'))
  
vdict

{'Black-grass': 78,
 'Charlock': 117,
 'Cleavers': 86,
 'Common Chickweed': 183,
 'Common wheat': 66,
 'Fat Hen': 142,
 'Loose Silky-bent': 196,
 'Maize': 66,
 'Scentless Mayweed': 154,
 'Shepherds Purse': 69,
 'Small-flowered Cranesbill': 148,
 'Sugar beet': 115}

In [44]:
##  DL starts here 
sz=224
PATH='/content/'
arch = resnet50
data = ImageClassifierData.from_paths(PATH,test_name='test',tfms=tfms_from_model(arch,sz))
learner = ConvLearner.pretrained(arch,data,precompute=True)
learner.fit(0.01,3)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:08<00:00, 11514075.10it/s]


100%|██████████| 13/13 [00:08<00:00,  2.17it/s]


epoch      trn_loss   val_loss   accuracy   
    0      1.217036   0.647664   0.809926  
    1      0.773832   0.505035   0.832101  
    2      0.579648   0.446555   0.853221  



[array([0.44656]), 0.853220697063579]

In [45]:
preds = learner.predict(is_test=True)
preds[:10]

array([[-10.0066 ,  -4.76048,  -2.96207,  -2.46832, -10.01618,  -0.16108, -10.31675,  -9.40278, -10.17634,
         -7.00804,  -6.208  ,  -7.39392],
       [ -3.60981,  -4.3863 ,  -4.56999,  -4.32637,  -2.84415,  -2.53009,  -1.28171,  -3.70998,  -0.82623,
         -3.52132,  -7.79895,  -3.52741],
       [ -6.84104,  -6.0169 ,  -5.1764 ,  -0.45068,  -7.30847,  -5.13274,  -7.10526,  -5.96178,  -4.89093,
         -1.65046,  -1.93766,  -8.67768],
       [ -9.30009,  -8.42372,  -6.14133,  -0.0312 ,  -8.78302,  -4.56519,  -9.51394,  -7.07452,  -5.83667,
         -5.36186,  -9.47977,  -4.70081],
       [-11.03787,  -0.01464,  -6.88102,  -6.60668,  -9.89922,  -5.46123, -11.81739,  -7.72825,  -9.85606,
         -6.54373,  -5.15194,  -9.0838 ],
       [ -8.55125,  -7.22553,  -8.38105,  -0.03989,  -7.81369,  -3.5688 , -10.15805,  -6.35504,  -5.95674,
         -5.90509,  -6.84431,  -6.71784],
       [-11.53064,  -3.61469,  -9.24436,  -7.66257,  -7.34509,  -6.19907, -12.23924,  -0.03123, -11.62433,

In [0]:
sample = pd.read_csv('sample_submission.csv')

In [49]:
sample.head()

,file,species
0,0021e90e4.png,Sugar beet
1,003d61042.png,Sugar beet
2,007b3da8b.png,Sugar beet
3,0086a6340.png,Sugar beet
4,00c47e980.png,Sugar beet


In [61]:
predictions = np.argmax(preds,axis=1)
preds_classes = [data.classes[i].replace("_", " ") for i in predictions]
preds_classes[:20]

['Fat Hen',
 'Scentless Mayweed',
 'Common Chickweed',
 'Common Chickweed',
 'Charlock',
 'Common Chickweed',
 'Maize',
 'Fat Hen',
 'Small-flowered Cranesbill',
 'Scentless Mayweed',
 'Loose Silky-bent',
 'Loose Silky-bent',
 'Loose Silky-bent',
 'Maize',
 'Common wheat',
 'Cleavers',
 'Loose Silky-bent',
 'Fat Hen',
 'Cleavers',
 'Fat Hen']

In [54]:
np.max(predictions)

11

In [55]:
labels = list(map(lambda x:train_data[x],predictions))
labels[:10]

['Shepherds Purse',
 'Sugar beet',
 'Common wheat',
 'Common wheat',
 'Loose Silky-bent',
 'Common wheat',
 'Common Chickweed',
 'Shepherds Purse',
 'Cleavers',
 'Sugar beet']

In [62]:
subm = pd.DataFrame({'file':test_data,'species':preds_classes})
subm.head()

,file,species
0,fd253a74e.png,Fat Hen
1,c06e7c748.png,Scentless Mayweed
2,98062cd87.png,Common Chickweed
3,060450d79.png,Common Chickweed
4,78750e0ff.png,Charlock


In [0]:
subm.to_csv('submission_1.csv',index=False)

In [64]:
!kaggle competitions submit -c plant-seedlings-classification -f submission_1.csv -m"plant seedlings vanilla resnet50 fixed classes"

Successfully submitted to Plant Seedlings Classification

In [71]:
## Improving score

lrf=learner.lr_find()

 80%|████████  | 48/60 [00:01<00:00, 42.06it/s, loss=0.588]


In [0]:
learner.sched.plot()

In [114]:
tfms = tfms_from_model(arch,sz,max_zoom=1.2,aug_tfms=transforms_top_down)
data = ImageClassifierData.from_paths(PATH,tfms=tfms,test_name='test')
learn = ConvLearner.pretrained(arch,data,precompute=True)
learn.fit(0.01,5)

100%|██████████| 23/23 [00:22<00:00,  2.47it/s]


epoch      trn_loss   val_loss   accuracy   
    0      1.243787   0.695337   0.799296  
    1      0.803093   0.521752   0.828873  
    2      0.697104   0.493334   0.851408  
    3      0.546973   0.467573   0.851408  
    4      0.46002    0.450979   0.852113  



[array([0.45098]), 0.8521126762242385]

In [115]:
learn.lr_find()

 85%|████████▍ | 45/53 [00:01<00:00, 33.06it/s, loss=0.71] 


In [0]:
learn.sched.plot()

In [117]:
learn.fit(1e-2,2)

epoch      trn_loss   val_loss   accuracy   
    0      0.335044   0.422633   0.870423  
    1      0.360429   1.088997   0.775352  



[array([1.089]), 0.7753521126760563]

In [118]:
learn.precompute=False
learn.fit(1e-2,3,cycle_len=1)

epoch      trn_loss   val_loss   accuracy   
    0      0.565684   0.438091   0.857746  
    1      0.532659   0.372565   0.878873  
    2      0.510764   0.370425   0.880986  



[array([0.37042]), 0.8809859154929578]

In [119]:
lr=1e-2
lr=np.array([1e-4,1e-3,1e-2])
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   accuracy   
    0      0.537331   0.373305   0.871127  
    1      0.601529   0.370897   0.883099  
    2      0.556279   0.343724   0.88662   
    3      0.561459   0.360869   0.880282  
    4      0.514086   0.335311   0.898592  
    5      0.493999   0.331136   0.892958  
    6      0.540614   0.329804   0.894366  



[array([0.3298]), 0.8943661971830986]

In [120]:
preds,y=learn.TTA()
accuracy(preds,y)

TypeError: ignored

In [0]:
preds = learn.predict(is_test=True)
preds = np.argmax(preds,axis=1)

In [87]:
data.classes

['Black-grass',
 'Charlock',
 'Cleavers',
 'Common Chickweed',
 'Common wheat',
 'Fat Hen',
 'Loose Silky-bent',
 'Maize',
 'Scentless Mayweed',
 'Shepherds Purse',
 'Small-flowered Cranesbill',
 'Sugar beet']

In [93]:
preds[:10]

array([5, 8, 3, 3, 1, 3, 7, 5, 3, 8])

In [122]:
labels = [data.classes[i] for i in preds]
labels[:10]

['Fat Hen',
 'Scentless Mayweed',
 'Common Chickweed',
 'Common Chickweed',
 'Charlock',
 'Common Chickweed',
 'Maize',
 'Fat Hen',
 'Common Chickweed',
 'Scentless Mayweed']

In [123]:
subm = pd.DataFrame({'file':test_data,'species':labels})
subm.head()

,file,species
0,fd253a74e.png,Fat Hen
1,c06e7c748.png,Scentless Mayweed
2,98062cd87.png,Common Chickweed
3,060450d79.png,Common Chickweed
4,78750e0ff.png,Charlock


In [0]:
subm.to_csv('submission_4.csv',index=False)

In [125]:
!kaggle competitions submit -c plant-seedlings-classification -f submission_4.csv -m"more data in validation set"

Successfully submitted to Plant Seedlings Classification

### Overfitting , so using labels and csv approach

In [126]:
!unzip -q train.zip

replace train/Fat Hen/5bdfa3a44.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [129]:
!ls train

Black-grass  Common Chickweed  Loose Silky-bent   Shepherds Purse
Charlock     Common wheat      Maize		  Small-flowered Cranesbill
Cleavers     Fat Hen	       Scentless Mayweed  Sugar beet


In [130]:
train_data

['Fat Hen',
 'Loose Silky-bent',
 'Black-grass',
 'Common wheat',
 'Small-flowered Cranesbill',
 'Shepherds Purse',
 'Charlock',
 'Common Chickweed',
 'Sugar beet',
 'Scentless Mayweed',
 'Cleavers',
 'Maize']

In [0]:
#shutil.rmtree('train')
#shutil.rmtree('train2')
!unzip -q train.zip
os.mkdir('train2')

In [0]:
main=[]
for data in train_data:
  images = os.listdir(f'train/{data}')
  classes=[]
  for image in images:
    shutil.move(f'train/{data}/{image}',f'train2/{image}')
    main.append([image,data])
  

In [0]:
labels = pd.DataFrame(main,columns=['file','species'])

In [0]:
labels.species=labels.species.apply(lambda x:x.replace(' ','_'))

In [181]:
labels.head()

,file,species
0,84097b0b7.png,Fat_Hen
1,52e6df585.png,Fat_Hen
2,43633755b.png,Fat_Hen
3,268e72cee.png,Fat_Hen
4,4a44a00eb.png,Fat_Hen


In [182]:

len(os.listdir('train2'))

4750

In [183]:
len(labels)

4750

In [0]:
labels.to_csv('labels.csv',index=False)

In [206]:
sz=224
arch = resnet34
tfms = tfms_from_model(arch,sz,aug_tfms=transforms_top_down,max_zoom=1.1)


data = ImageClassifierData.from_csv(PATH,folder='train2',
                                    csv_fname=f'{PATH}labels.csv',tfms=tfms,
                                    val_idxs=get_cv_idxs(n=len(labels)),
                                    test_name='test',bs=64)


learn = ConvLearner.pretrained(arch,data,precompute=True)
learn.fit(0.01,5)

epoch      trn_loss   val_loss   accuracy   
    0      1.54493    0.910519   0.701053  
    1      1.108384   0.716374   0.772632  
    2      0.899323   0.65237    0.795789  
    3      0.755432   0.599043   0.804211  
    4      0.680614   0.562124   0.813684  


[array([0.56212]), 0.8136842115301834]

In [207]:
learn.fit(1e-2,3)

epoch      trn_loss   val_loss   accuracy   
    0      0.619655   0.529351   0.828421  
    1      0.575808   0.514659   0.834737  
    2      0.549378   0.517393   0.826316  


[array([0.51739]), 0.8263157892227173]

In [208]:
learn.precompute=False
learn.fit(1e-2,3,cycle_len=1)

epoch      trn_loss   val_loss   accuracy   
    0      0.738532   0.514069   0.830526  
    1      0.750635   0.497683   0.835789  
    2      0.734871   0.488725   0.841053  


[array([0.48872]), 0.8410526313279805]

In [209]:
learn.unfreeze()

lrs=np.array([1e-4,1e-3,1e-2])
learn.fit(lrs,3,cycle_len=1,cycle_mult=2)

epoch      trn_loss   val_loss   accuracy   
    0      0.727722   0.387464   0.871579  
    1      0.496754   0.274136   0.909474  
    2      0.387701   0.266621   0.910526  
    3      0.322982   0.223491   0.933684  
    4      0.266132   0.208089   0.927368  
    5      0.236686   0.198078   0.934737  
    6      0.214451   0.194837   0.934737  


[array([0.19484]), 0.9347368421052632]

In [210]:
learn.fit(lrs,3,cycle_len=1,cycle_mult=2)

epoch      trn_loss   val_loss   accuracy   
    0      0.219997   0.199913   0.932632  
    1      0.203981   0.206977   0.930526  
    2      0.190465   0.186175   0.945263  
    3      0.185506   0.179897   0.948421  
    4      0.157056   0.178806   0.938947  
    5      0.154795   0.171361   0.941053  
    6      0.141548   0.175372   0.943158  


[array([0.17537]), 0.9431578947368421]

In [157]:
preds,y = learn.TTA()

In [0]:
preds = learn.predict(is_test=True)

In [213]:
preds = np.argmax(preds,axis=1)
labelset = [data.classes[i].replace('_',' ') for i in preds]
labelset[:10]

['Cleavers',
 'Scentless Mayweed',
 'Common Chickweed',
 'Common Chickweed',
 'Charlock',
 'Common Chickweed',
 'Maize',
 'Fat Hen',
 'Common Chickweed',
 'Scentless Mayweed']

In [214]:
subm = pd.DataFrame({'file':test_data,'species':labelset})
subm.head()

,file,species
0,fd253a74e.png,Cleavers
1,c06e7c748.png,Scentless Mayweed
2,98062cd87.png,Common Chickweed
3,060450d79.png,Common Chickweed
4,78750e0ff.png,Charlock


In [0]:
subm.to_csv('submission_7.csv',index=False)

In [216]:
!kaggle competitions submit -c plant-seedlings-classification -f submission_7.csv -m"final submission resnet34"

Successfully submitted to Plant Seedlings Classification